<h1>幻想篮球 第二部分</h1>

<h2>简介</h2>

如果你熟悉幻想篮球并完成了我们的**第一个示例**，你可能注意到我们解决的阵容实际上并不符合许多流行比赛的标准。第一个示例是作为数学优化的基础示例，所以请确保先完成那个示例以了解问题背景。此外，我们开发了一个预测模型来预测每个球员的幻想分数，我们将再次使用该预测（不再重复建模过程）。

在这个后续示例中，我们将扩展优化模型以满足典型的 [DraftKings](https://www.draftkings.com/help/rules/nba) 阵容要求。具体来说，我们需要对模型进行以下更改：
- 允许某些球员可以被选择填充最多两个位置（例如勒布朗·詹姆斯可以填充大前锋或小前锋位置）
- 将阵容从5人增加到8人
- 确保三个新的阵容位置包括一个后卫（得分后卫或控球后卫）、一个前锋（小前锋或大前锋）和一个通用球员（任何位置）

简单回顾一下前一个示例中我们看到的内容：在选择5人阵容时，有*很多*可能性；保持在工资帽下并不那么容易；实施直观的决策规则可能导致次优阵容 -- 这让我们瞥见了为什么数学优化应该成为每个人分析工具包的一部分。

**本项目的代码库可以通过以下链接访问：**


<h2>目标和先决条件</h2>

与第一个示例一样，这里的目标是基于前一个示例的发现，选择NBA（美国职业篮球联赛）最佳阵容，以产生最高的幻想总分，这些分数由球员的比赛数据组成。

如上所述，我们将组建一支符合DraftKings比赛规则的阵容。当选择一支包含每个位置各一名球员的5人球队时，有超过200万种可能的阵容。你能计算出这个示例中可能的阵容数量吗？提示：数量非常大。请记住，有25名控球后卫、23名小前锋、22名得分后卫、19名大前锋和9名中锋。答案（近似值）将在稍后给出。

同样如前所述，本示例将使用与我们幻想篮球初学者示例相同的预测结果，该示例使用2016-2017和2017-2018赛季的历史数据来预测每个球员在2017年12月25日比赛中的幻想得分。

本示例假设您有使用Python进行数据操作的经验，并需要安装以下软件包：

- **pandas**：用于数据分析和操作
- **math**：用于数学运算
- **gurobipy**：用于使用Gurobi构建和求解优化问题

我们还将探索几种不同的方式来编写gurobipy中的求和和约束，这样你可以找到最适合你的方式。注意：任何类似于<gurobi.Constr \*Awaiting Model Update\*>的输出都可以忽略，并且添加了分号以抑制该输出。

<h2>问题陈述和解决方法</h2>

通过构建一个允许某些球员被分配到多个位置的模型，并且还扩展了阵容，我们使模型变得更加复杂。

我们的最终阵容需要包括以下每个位置：
- 控球后卫 (PG)
- 得分后卫 (SG)
- 小前锋 (SF)
- 大前锋 (PF)
- 中锋 (C)
- 后卫 (SG,PG)
- 前锋 (SF,PF)
- 通用 (PG, SG, SF, PF, C)

问题的解决方案包括两个部分：1）**幻想分数预测**和2）**阵容优化**。

我们将首先加载一个包含合格球员的数据集，显示他们的潜在位置、薪水以及他们即将到来的表现预测。这些信息将作为我们优化模型的输入，确保我们满足DraftKings比赛的规则，同时最大化我们团队的幻想总分。

<h3>幻想分数预测</h3>

这一部分将很简短，因为我们使用的是第一个示例中的预测模型的输出。我们会看到在这个版本中，一些球员有一个*主要位置*和*替代位置*，这将使球员在符合条件的情况下填充不同的位置。例如，詹姆斯·哈登可以填充控球后卫或得分后卫位置，而拉塞尔·威斯布鲁克只能作为控球后卫。

我们首先加载解决问题所需的库，并导入第一个问题部分中的预测模型数据。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd                                       #导入pandas
import math                                               #导入math
import gurobipy as gp                                     #导入Gurobi

In [2]:
# player_predictions = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/fantasy_basketball/results_target_advanced.csv')     #加载处理过的数据集
player_predictions = pd.read_csv('results_target_advanced.csv')     #加载处理过的数据集
player_predictions.sort_values(by='PredictedFantasyPoints',ascending=False).head(20)

,Player,MainPosition,AlternativePosition,Team,Opp,Salary,PredictedFantasyPoints,Points/Salary Ratio
4,Joel Embiid,C,NaN,PHI,NYK,9500,51.313689,5.401441
0,James Harden,PG,SG,HOU,OKC,11100,48.809577,4.397259
1,LeBron James,SF,PF,CLE,GSW,11000,48.149718,4.377247
2,Russell Westbrook,PG,NaN,OKC,HOU,10900,44.007224,4.037360
3,Kevin Durant,SF,PF,GSW,CLE,10500,43.438575,4.137007
19,Dario Saric,PF,C,PHI,NYK,6200,40.505486,6.533143
5,Ben Simmons,PG,SF,PHI,NYK,9300,38.692817,4.160518
12,Kyle Kuzma,SF,PF,LAL,MIN,7300,38.201774,5.233120
8,Jimmy Butler,SG,NaN,MIN,LAL,8400,37.873164,4.508710
13,Draymond Green,PF,C,GSW,CLE,7200,37.018949,5.141521


你已经算出可能的阵容总数了吗？大约是 $3.6 \times 10^{11}$，这非常多。

这就是数学优化和Gurobi最适合的地方：高效地探索庞大的决策空间，并为最佳决策提供急需的工具。

<h3>最佳DraftKings阵容选择</h3>

在设置我们的优化模型时，我们首先需要进行一些定义。有些定义与之前相同，但由于这个示例更复杂，我们需要在某些定义上更加详细。

**集合和索引**

$i$ 是所有球员的索引

$j$ 是篮球位置的索引（PG,SG,SF,PF,C）

**输入参数**

$p_{i}$：球员 $i$ 的预测幻想分数

$s_{i}$：球员 $i$ 的薪水

$S$：我们的总可用薪水

In [ ]:
players = player_predictions["Player"].tolist()
positions = player_predictions["MainPosition"].unique().tolist()
salaries = player_predictions["Salary"].tolist()
fantasypoints = player_predictions["PredictedFantasyPoints"].tolist()
S = 50000

salary_dict = {players[i]: salaries[i] for i in range(len(players))}
points_dict = {players[i]: fantasypoints[i] for i in range(len(players))}

m = gp.Model()

Set parameter LicenseID to value 2601452


**决策变量**

由于某些球员可以填充一个或两个位置，我们需要将每个球员映射到他们的合格位置。此外，我们需要将位置索引添加到我们的决策变量中。与二元变量（即仅取值为0或1的变量）$y_i$不同，我们有$y_{i,j}$。

$y_{i,j}$：如果球员 $i$ 被选择在位置 $j$，则该变量等于1；否则为0。

In [7]:
mainposition = list(zip(player_predictions.Player, player_predictions.MainPosition))
alternativeposition = list(zip(player_predictions.Player, player_predictions.AlternativePosition))
indices = mainposition + alternativeposition
player_pos_map = [t for t in indices if not any(isinstance(n, float) and math.isnan(n) for n in t)]

y = m.addVars(player_pos_map, vtype=gp.GRB.BINARY, name="y")

**目标函数**

我们问题的目标函数是最大化我们阵容的总幻想分数，与上次相同，但使用不同索引的决策变量。

\begin{align}
 Max \hspace{0.2cm} Z = \sum_{i,j} p_{i} \cdot y_{i,j}
\end{align}

In [12]:
m.setObjective(gp.quicksum(points_dict[i]*y[i,j] for i,j in player_pos_map), gp.GRB.MAXIMIZE)

**约束条件**

我们的模型仍然需要填充每个主要篮球位置（PG, SG, SF, PF, C）。上次我们要求每个位置恰好一个，所以这些约束是等式（注意这也意味着我们的阵容大小为五人）。在这个版本的模型中，我们需要*至少一个*每个位置，因为可能只使用一个大前锋（PF）。

对于每个位置 $j$：
\begin{align}
\sum_{i} y_{i,j} \geq 1
\end{align}

这里我们有几种添加这些约束的选项。第一种是直观地遍历位置集合。第二种使用稍微不同的函数来添加约束（*addConstr* -> *addConstrs*）。这个函数将for循环直接作为参数。**运行两者会在模型中重复添加约束，这不是最佳实践，所以请查看每种方法并在运行单元格之前注释掉其中一个。**

In [13]:
# 编写上述约束的选项1
for j in positions:
    m.addConstr(gp.quicksum(y[i,j] for i, pos in player_pos_map if pos==j)>=1, name = "pos" + j)

# # 选项2，一种稍微紧凑的方式来添加相同的约束集
# m.addConstrs((gp.quicksum(y[i,j] for i, pos in player_pos_map if pos==j)>=1 for j in positions), name = "pos");

现在让我们处理新阵容的额外位置：后卫、前锋和通用位置。这里值得一提的是，在数学优化中有不同的方法来构建相同的问题。一些方法可能比其他方法更好，编写高效的模型将成为你解决更大更复杂问题时的重要技能。

首先考虑后卫位置的额外位置，可以由PG或SG填充。考虑到我们要创建的整体阵容，我们已经保证了一个PG和一个SG（所以总共两个后卫）。为了确保我们得到一个额外的后卫，需要添加一个约束，要求总后卫数*至少*为三个。

\begin{align}
\sum_{i} y_{i,j} \geq 3, 位置\space j \space 是\space PG\space 或 \space SG
\end{align}

同样需要对前锋（SF或PF）进行处理
\begin{align}
\sum_{i} y_{i,j} \geq 3, 位置\space j \space 是\space SF\space 或 \space PF
\end{align}

这里使用不等式很重要，因为我们稍后会处理通用位置。

In [14]:
m.addConstr(gp.quicksum(y[i,j] for i, j in player_pos_map if (j=='PG' or j=='SG'))>=3)
m.addConstr(gp.quicksum(y[i,j] for i, j in player_pos_map if (j=='SF' or j=='PF'))>=3);

现在我们有了决策变量的职位索引，我们需要确保每个球员被分配到*最多*一个位置（主要位置或替代位置，但不能同时）。为此，我们对每个球员的每个位置 $j$ 进行求和，并将该求和限制为一个。

对于每个球员 $i$，
\begin{align}
\sum_{j} y_{i,j} \leq 1
\end{align}

这里我们将使用另一种方法来对变量的一个索引求和，即在末尾添加*.sum*并用"\*"替换我们要求和的索引。这很有用，因为每个球员并不适合每个位置，这种语法会自动对第二个索引求和。

In [15]:
m.addConstrs((y.sum(i, "*") <= 1 for i in players), name="max_one");

一个好的练习是使用这种语法重写其他求和，并尝试使用quicksum编写上述约束。

到目前为止，我们已经解决了八个阵容位置中的七个。由于通用位置可以有任何位置，我们只需要通过将所有球员和位置的总和设置为八来要求选择的球员总数为八。

\begin{align}
\sum_{i,j} y_{i,j} = 8
\end{align}

In [16]:
m.addConstr(gp.quicksum(y[i,j] for i,j in player_pos_map) == 8, name="full_lineup");

最后，我们需要保持在工资帽 $S$ 下，在典型的DraftKings比赛中为 $\$50,000$：

\begin{align}
\sum_{i,j} s_{i} \cdot y_{i,j} \leq S
\end{align}

In [17]:
cap = m.addConstr(gp.quicksum(salary_dict[i]*y[i,j] for i,j in player_pos_map) <= S, name="salary")

在这个最后的约束中，我们存储它以便在模型运行后轻松获取有关此约束的信息，并且模型会像我们没有存储约束一样更新。现在是时候找到最佳阵容了。

In [18]:
m.optimize()  # 优化我们的模型

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 107 rows, 167 columns and 808 nonzeros
Model fingerprint: 0xc60db15e
Variable types: 0 continuous, 167 integer (167 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [7e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 206.2597683
Presolve removed 29 rows and 0 columns
Presolve time: 0.00s
Presolved: 78 rows, 167 columns, 779 nonzeros
Variable types: 0 continuous, 167 integer (167 binary)

Root relaxation: objective 2.864073e+02, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

注意上面的输出显示我们有167个二元变量，这远少于 $98 \cdot 5 = 490$ 个变量，如果我们将所有球员映射到所有篮球位置。 此外，我们需要更多的约束来消除不合格的球员/位置组合。 虽然这在这个小示例中不会有太大区别，但编写高效模型是数学优化中的一项宝贵技能。

让我们显示我们的最佳阵容。

In [19]:
player_selections = []
for v in m.getVars():
    if (abs(v.x) > 1e-6):
        player_selections.append(tuple(y)[v.index])
        
df = pd.DataFrame(player_selections, columns = ['Player','Assigned Position'])
df = df.merge(pd.DataFrame(list(salary_dict.items()), columns=['Player', 'Salary']), left_on=['Player'], right_on=['Player'])
lineup = df.merge(pd.DataFrame(list(points_dict.items()), columns=['Player', 'Predicted Points']), left_on=['Player'], right_on=['Player'])

lineup.sort_values(by=['Assigned Position'])

,Player,Assigned Position,Salary,Predicted Points
0,Joel Embiid,C,9500,51.313689
6,Draymond Green,C,7200,37.018949
2,Dario Saric,PF,6200,40.505486
4,Jordan Bell,PF,4900,33.083296
3,Jeff Teague,PG,6000,31.460451
5,Jarrett Jack,PG,4600,27.780012
1,Kyle Kuzma,SF,7300,38.201774
7,Josh Hart,SG,3700,24.262564


In [20]:
print('Total fantasy score: ', round(m.objVal,2))
print('Remaining salary: ', cap.Slack)

Total fantasy score:  283.63
Remaining salary:  600.0


最后一个打印语句使用了我们之前存储的约束。 约束的*Slack*属性将显示不等式两边的任何差距。 对于这个应用程序，它是工资帽和最佳阵容中使用的工资之间的差距，显示有 $600 的未使用工资。

In [21]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment


<h2>结论</h2>

在这个notebook中，我们完成了一个从包含NBA球员数据的原始数据集开始，并最终得出一个最佳幻想篮球阵容的问题。

具体来说，在这个部分中，我们：
- 扩展了初始模型以反映创建幻想篮球阵容的真实复杂性
- 发现了多种向模型添加约束集的方法
- 使用了两种求和命令
- 使用模型的属性获取有关最佳解决方案的更多信息

总的来说，这个两部分的示例展示了即使有最好的预测模型，做出最佳决策仍然是一个复杂的过程。除了机器学习技术，数学优化是一个完善的分析工具箱中不可或缺的一部分。